# import 

In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pickle

# 取得先

In [2]:
with open("check_URLs.txt", "r") as f:
    check_URLS = [line.replace("\n", "") for line in f]
check_URLS

['https://www.aoyama.ac.jp/president_room/covid19',
 'https://www.aoyama.ac.jp/admission/undergraduate/flow/freshman.html']

In [3]:
with open("must_check_URLs.txt", "r") as f:
    must_URLS = [line.replace("\n", "") for line in f]
must_URLS

['https://www.aoyama.ac.jp/emergency/']

# 取得済み情報の読み込み

In [14]:
with open("information.pkl", "rb") as f:
    information = pickle.load(f)

# 日付と text と URL を取得する関数

In [5]:
def get_relation(content):
    text = content.text
    if not content.has_attr("href"):
        return False
    url = content["href"]
    if "https://www.aoyama.ac.jp" in url:
        pass    
    elif url[:4] != "http" or url[:3] == "#id":
        return False
    else:
        url = "https://www.aoyama.ac.jp" + url
    date = re.search(r'[0-9]*\.[0-9]+\.[0-9]+', text)
    if date is not None:
        date = date.group().replace(".", "/")
    else:
        date = re.search(r'[0-9]*.[0-9]+月[0-9]+日', text)
        if date is not None:
            date = date.group()
            # 正規表現ゆるゆるなので（4月などの（が入るかも
            if date[0] not in set("0123456789"):
                date = date[1:]            
            if "年" not in date:
                date = "2020/"+date
            else:
                date = date.replace("年", "/")
            # 面倒なのでごり押し
            for r, n in [["/"+str(i)+"月", "/0"+str(i)+"/"] for i in range(1, 10)]:
                date = date.replace(r, n)
            for r, n in [["/"+str(i)+"日", "/0"+str(i)] for i in range(1, 10)]:
                date = date.replace(r, n)
            date = date.replace("月", "/").replace("日", "")
            
    if date is None:
        return False
    
    if date not in information:
        information[date] = [{"text":text, "url":url}]
    else:
        information[date].append({"text":text, "url":url})
    #print(date, text, url)

In [6]:
def check_must(url):
    for must in must_URLS:
        if must in url:
            return True
    return False

# get_emergency
赤いやつを get する

In [8]:
html = requests.get(check_URLS[0])

In [9]:
soup = BeautifulSoup(html.text, "html.parser")
items = soup.findAll("a")
items = [item for item in items if item.has_attr("href") and check_must(item["href"])]
items

[<a class="text" href="https://www.aoyama.ac.jp/emergency/20200427_01"><p class="emergency-text"><span class="emergency-date">2020.04.27：</span>【重要・緊急】学生への経済的支援について</p></a>,
 <a class="text" href="https://www.aoyama.ac.jp/emergency/20200407_01"><p class="emergency-text"><span class="emergency-date">2020.04.14：</span>【重要・緊急】新型コロナウイルス感染症（COVID-19）の拡大に伴う入構禁止措置について（2020.04.14更新）</p></a>,
 <a class="text" href="https://www.aoyama.ac.jp/emergency/20200409"><p class="emergency-text"><span class="emergency-date">2020.04.09：</span>新型コロナウイルス感染症（COVID-19）に関するお知らせ（まとめ）(2020.04.25更新）</p></a>]

In [10]:
[get_relation(content) for content in items]

[None, None, None]

# check_URLs
対象の url から情報を get

In [11]:
for c_URL in check_URLS:
    html = requests.get(c_URL)
    soup = BeautifulSoup(html.text, "html.parser")
    items = soup.findAll("div", attrs=["section_inner"])[0]
    items = items.findAll()
    items = [item.findAll("a") for item in items]
    items = [item for item in items if len(item) != 0]
    items = list(set([info for item in items for info in item]))
    [get_relation(content) for content in items]
    items = soup.findAll("li", attrs=["subject_item"])
    items = [item.findAll("a") for item in items]
    items = [item for item in items if len(item) != 0]
    items = list(set([info for item in items for info in item]))
    [get_relation(content) for content in items]

In [12]:
information

{'2020/03/03': [{'text': '\n2019年度卒業礼拝中止のお知らせ（2020年3月3日掲出）',
   'url': 'https://www.aoyama.ac.jphttps://www.aoyamagakuin.jp/post_consistency/graduationservice/'}],
 '2020/03/23': [{'text': 'TOEIC IP・TOEFL ITPテストの実施変更について（3月23日更新）',
   'url': 'https://www.aoyama.ac.jp/faculty/freshman/englishtest.html'}],
 '2020/04/03': [{'text': '留学・国際交流に関すること（4月3日更新）',
   'url': 'https://www.aoyama.ac.jphttp://www.iec.aoyama.ac.jp/'}],
 '2020/04/06': [{'text': 'ITオリエンテーションに関すること（4月6日更新）',
   'url': 'https://www.aoyama.ac.jphttps://www.aim.aoyama.ac.jp/readme-st/rm-s-fm/it-orientation/'},
  {'text': '現代経済デザイン学科（4月6日更新）',
   'url': 'https://www.aoyama.ac.jphttps://cdn.aoyama.ac.2xx.jp/wp-content/uploads/2020/04/122_a-1.pdf'},
  {'text': '法学科（4月6日更新）',
   'url': 'https://www.aoyama.ac.jphttps://cdn.aoyama.ac.2xx.jp/wp-content/uploads/2020/04/131_a-1.pdf'},
  {'text': '経営学科・マーケティング学科（4月6日更新）',
   'url': 'https://www.aoyama.ac.jphttps://cdn.aoyama.ac.2xx.jp/wp-content/uploads/2020/04/141_a-1.pdf'},
  {'tex

In [13]:
with open("information.pkl", "wb") as f:
    pickle.dump(information, f)

In [19]:
import json
#json 化
with open("information.json", "w", encoding="utf8") as f:
    infor
    json.dump(information, f,ensure_ascii=False)

In [18]:
?json.dump